In [27]:
import json, openai
import configparser
import warnings
warnings.filterwarnings('ignore')
import uuid
from time import sleep

In [3]:
credentials = configparser.ConfigParser()
credentials.read_file(open('apidata.config'))

In [25]:
my_key = credentials['OPENAI']['KEY']
my_org = credentials['OPENAI']['org']

In [6]:
#Lets begin

In [8]:
places = ['India','United States of America',
         'Pakistan','Russia']
year = [2022, 2024, 2026, 2028]
ideas1 = ['Power','Ability','automation','danger']
ideas2 = ['Government','education','society','space faring']

In [11]:
str(uuid.uuid4())

'43503acf-9a24-4a3d-81ad-67c6184aabc3'

In [106]:
prompt_list = []
for plc in places:
    for yr in year:
        for id1 in ideas1:
            for id2 in ideas2:
                prompt =f"""{uuid.uuid4()}
Imagine you are the highest earning youtube influencer.
The year is {yr} and you are living in {plc}.You are asked to influence the 
masses on topic {id1}. Write 5 points each 100 words elaborating {id1} impact on {id2}
so it can be made in to a video"""
                prompt_list.append(prompt)

In [103]:
prompt_parts = []
for plc in places:
    for yr in year:
        for id1 in ideas1:
            for id2 in ideas2:
                prompt_part =f"""Year: {yr}\nPlace: {plc}\nIdea1:{id1}\nIdea2:{id2}"""
                prompt_parts.append(prompt_part)

In [107]:
len(prompt_list)

256

In [108]:
len(prompt_parts)

256

In [109]:
prompt_list[0:10]

['dfb4d4c0-ca1a-4061-bc1d-cc7e8d3a8f66\nImagine you are the highest earning youtube influencer.\nThe year is 2022 and you are living in India.You are asked to influence the \nmasses on topic Power. Write 5 points each 100 words elaborating Power impact on Government\nso it can be made in to a video',
 'c19bfe5b-ff57-4bfe-a3ac-4ed2c761e094\nImagine you are the highest earning youtube influencer.\nThe year is 2022 and you are living in India.You are asked to influence the \nmasses on topic Power. Write 5 points each 100 words elaborating Power impact on education\nso it can be made in to a video',
 '479e0bc3-7ea6-487a-a364-0904c9a26a9b\nImagine you are the highest earning youtube influencer.\nThe year is 2022 and you are living in India.You are asked to influence the \nmasses on topic Power. Write 5 points each 100 words elaborating Power impact on society\nso it can be made in to a video',
 '0857583c-9800-4324-8839-ba36f9a93664\nImagine you are the highest earning youtube influencer.\nT

In [110]:
prompt_parts[0:10]

['Year: 2022\nPlace: India\nIdea1:Power\nIdea2:Government',
 'Year: 2022\nPlace: India\nIdea1:Power\nIdea2:education',
 'Year: 2022\nPlace: India\nIdea1:Power\nIdea2:society',
 'Year: 2022\nPlace: India\nIdea1:Power\nIdea2:space faring',
 'Year: 2022\nPlace: India\nIdea1:Ability\nIdea2:Government',
 'Year: 2022\nPlace: India\nIdea1:Ability\nIdea2:education',
 'Year: 2022\nPlace: India\nIdea1:Ability\nIdea2:society',
 'Year: 2022\nPlace: India\nIdea1:Ability\nIdea2:space faring',
 'Year: 2022\nPlace: India\nIdea1:automation\nIdea2:Government',
 'Year: 2022\nPlace: India\nIdea1:automation\nIdea2:education']

In [41]:
str(uuid.uuid1()).split('-')[0]

'af9315d6'

In [126]:
def gpt3_completion(org, key,prompt,prompt_part,
                    engine='text-davinci-002',temp=1.0,
                   top_p=1.0,tokens=500,freq_pen=0.0,
                   pres_pen=0.0,stop=['asdfasdf','asdfasdf'],
                   filename='temp_file.jsonl'):
    """The function will take the prompt and the prompt parts to 
    begin the data collection. Once the data collection is completed, 
    the prompt part will be used for fine tuning. The prompt used will 
    be discarded"""
    openai.api_key = key
    openai.organization = org
    max_retry=5
    retry = 0
    prompt = prompt.encode(encoding='ASCII',errors='ignore').decode()
    while True:
        try:
            response = openai.Completion.create(
            model=engine,
            prompt=prompt,
            temperature=temp,
            max_tokens=tokens,
            top_p=top_p,
            frequency_penalty=freq_pen,
            presence_penalty=pres_pen,
            stop=stop)
            
            text = response['choices'][0]['text'].strip()
            outprompt = prompt_part
            dict_reply = {'prompt':outprompt,'completion':text}
            with open(filename,'a+') as taker:
                json.dump(dict_reply,fp=taker)
                taker.write('\n')
            
            return dict_reply
        except Exception as e:
            retry += 1
            if retry > max_retry:
                return f'GPT errored out {e}'
            sleep(1)

In [127]:
dict_to_work = gpt3_completion(org=my_org,key=my_key,
                prompt=prompt_list[0],
                prompt_part=prompt_parts[0],
               filename='updated_payload.jsonl')

In [128]:
dict_to_work


{'prompt': 'Year: 2022\nPlace: India\nIdea1:Power\nIdea2:Government',
 'completion': 'for Youtube\n\n\n1. Power has a massive impact on governments across the world.\n\n2. In India, power is a key factor in determining the success of the government.\n\n3. The Indian government has been able to use power to its advantage in recent years.\n\n4. Power is a major tool that can be used by governments to achieve their goals.\n\n5. India is a key player in the global power market and its government’s policies have a direct impact on the world.'}

In [96]:
!cat updated_payload.jsonl

{"prompt": "Year: 2022\nPlace: 2022\nIdea1:Power\nIdea2:Government", "completion": ".\n\n\n1. India is a democracy and power rests with the people.\n\n2. The government is accountable to the people and needs to exercise power responsibly.\n\n3. Power should be used to further the common good and not for personal gain.\n\n4. Those in positions of power should be role models for the rest of society.\n\n5. The use of power should be transparent and accountable to the people."}


In [111]:
for ind,pmt in enumerate(prompt_list[0:20]):
    #print(prompt_list[ind])
    gpt3_completion(org=my_org,key=my_key,
                prompt=prompt_list[ind],
                   prompt_part=prompt_parts[ind],
                    filename='model_payload.jsonl')

In [112]:
!cat model_payload.jsonl

{"prompt": "Year: 2022\nPlace: India\nIdea1:Power\nIdea2:Government", "completion": "by you .\n\n\n\n1. In a democracy, power ultimately lies with the people.\n\n2. The government is elected by the people and is accountable to them.\n\n3. The government needs the people\u2019s consent to exercise its power.\n\n4. The people have the right to protest and overthrow the government if it becomes tyrannical.\n\n5. Therefore, the government must use its power wisely and for the benefit of the people.\n\n1. The government has the power to make laws that govern the people.\n\n2. The government has the power to enforce those laws.\n\n3. The government has the power to tax the people.\n\n4. The government has the power to spend the people\u2019s money.\n\n5. Therefore, the government must be responsible and accountable for its actions."}
{"prompt": "Year: 2022\nPlace: India\nIdea1:Power\nIdea2:education", "completion": "1.\tPower is the ability to influence others.\n\n2.\tPower can be used to ma

In [64]:
def file_upload(filename, purpose='fine-tune'):
    resp = openai.File.create(purpose=purpose,file=open(file=filename,
                                                        mode='rb'))
    print(resp)
    return resp

In [113]:
file_upload(filename='model_payload.jsonl')

{
  "bytes": 13403,
  "created_at": 1677997340,
  "filename": "file",
  "id": "file-3FeGCNbpkIgtcOtatfOSYzdO",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}


<File file id=file-3FeGCNbpkIgtcOtatfOSYzdO at 0x7f56e096f6a0> JSON: {
  "bytes": 13403,
  "created_at": 1677997340,
  "filename": "file",
  "id": "file-3FeGCNbpkIgtcOtatfOSYzdO",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}

In [116]:
def file_list():
    #print(openai.File.list())
    return openai.File.list()

In [69]:
openai.FineTune.create(training_file="file-16SdoYlLlW2IRQEykCWwjvWv")

<FineTune fine-tune id=ft-emvHXGEmz5Zj0Hk0iTyDfsdq at 0x7f56e1d4ebb0> JSON: {
  "created_at": 1677994116,
  "events": [
    {
      "created_at": 1677994116,
      "level": "info",
      "message": "Created fine-tune: ft-emvHXGEmz5Zj0Hk0iTyDfsdq",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-emvHXGEmz5Zj0Hk0iTyDfsdq",
  "model": "curie",
  "object": "fine-tune",
  "organization_id": "org-QWWIpNe4Z3KsRVI0YR6lhyJW",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 6195,
      "created_at": 1677993663,
      "filename": "file",
      "id": "file-16SdoYlLlW2IRQEykCWwjvWv",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    }
  ],
  "updated_at": 1677994116,
  "validation_files": []
}

In [70]:
import requests

In [117]:
def finetune_model(fileId, suffix,model='davinci',api_key=my_key):
    header = {'Content-Type':'application/json',
              'Authorization':f'Bearer {api_key}'}
    payload = {'training_file':fileId,'model':model,'suffix':suffix}
    resp = requests.request(method='POST',
                            url='https://api.openai.com/v1/fine-tunes',
                           headers=header,
                           json=payload,timeout=40)
    print(resp.json())

In [118]:
def finetune_event(ftId,api_key=my_key):
    header = {'Content-Type':'application/json',
              'Authorization':f'Bearer {api_key}'}
    resp = requests.request(method='GET',
                url=f'https://api.openai.com/v1/fine-tunes/{ftId}/events',
                           headers=header,timeout=40)
    print(resp.json())

In [124]:
def finetune_get(ftId,api_key=my_key):
    header = {'Content-Type':'application/json',
              'Authorization':f'Bearer {api_key}'}
    resp = requests.request(method='GET',
                url=f'https://api.openai.com/v1/fine-tunes/{ftId}',
                           headers=header,timeout=40)
    print(resp.json())

In [85]:
finetune_model(fileId='file-16SdoYlLlW2IRQEykCWwjvWv',
               suffix='influencer_generator',api_key=my_key)

{'object': 'fine-tune', 'id': 'ft-iUuWQgOAPtb31KnOmo8emkiv', 'hyperparams': {'n_epochs': 4, 'batch_size': None, 'prompt_loss_weight': 0.01, 'learning_rate_multiplier': None}, 'organization_id': 'org-QWWIpNe4Z3KsRVI0YR6lhyJW', 'model': 'davinci', 'training_files': [{'object': 'file', 'id': 'file-16SdoYlLlW2IRQEykCWwjvWv', 'purpose': 'fine-tune', 'filename': 'file', 'bytes': 6195, 'created_at': 1677993663, 'status': 'processed', 'status_details': None}], 'validation_files': [], 'result_files': [], 'created_at': 1677994822, 'updated_at': 1677994822, 'status': 'pending', 'fine_tuned_model': None, 'events': [{'object': 'fine-tune-event', 'level': 'info', 'message': 'Created fine-tune: ft-iUuWQgOAPtb31KnOmo8emkiv', 'created_at': 1677994822}]}


In [119]:
finetune_model(fileId='file-3FeGCNbpkIgtcOtatfOSYzdO',
               suffix='video_idea_generator',api_key=my_key)

{'object': 'fine-tune', 'id': 'ft-x4oK2iIg4uY3hoBZEVaaTtZm', 'hyperparams': {'n_epochs': 4, 'batch_size': None, 'prompt_loss_weight': 0.01, 'learning_rate_multiplier': None}, 'organization_id': 'org-QWWIpNe4Z3KsRVI0YR6lhyJW', 'model': 'davinci', 'training_files': [{'object': 'file', 'id': 'file-3FeGCNbpkIgtcOtatfOSYzdO', 'purpose': 'fine-tune', 'filename': 'file', 'bytes': 13403, 'created_at': 1677997340, 'status': 'processed', 'status_details': None}], 'validation_files': [], 'result_files': [], 'created_at': 1677997477, 'updated_at': 1677997477, 'status': 'pending', 'fine_tuned_model': None, 'events': [{'object': 'fine-tune-event', 'level': 'info', 'message': 'Created fine-tune: ft-x4oK2iIg4uY3hoBZEVaaTtZm', 'created_at': 1677997477}]}


In [87]:
finetune_event(ftId='ft-iUuWQgOAPtb31KnOmo8emkiv',api_key=my_key)

{'object': 'list', 'data': [{'object': 'fine-tune-event', 'level': 'info', 'message': 'Created fine-tune: ft-iUuWQgOAPtb31KnOmo8emkiv', 'created_at': 1677994822}, {'object': 'fine-tune-event', 'level': 'info', 'message': 'Fine-tune costs $0.17', 'created_at': 1677995041}, {'object': 'fine-tune-event', 'level': 'info', 'message': 'Fine-tune enqueued. Queue number: 0', 'created_at': 1677995042}, {'object': 'fine-tune-event', 'level': 'info', 'message': 'Fine-tune started', 'created_at': 1677995043}, {'object': 'fine-tune-event', 'level': 'info', 'message': 'Completed epoch 1/4', 'created_at': 1677995159}, {'object': 'fine-tune-event', 'level': 'info', 'message': 'Completed epoch 2/4', 'created_at': 1677995162}, {'object': 'fine-tune-event', 'level': 'info', 'message': 'Completed epoch 3/4', 'created_at': 1677995165}, {'object': 'fine-tune-event', 'level': 'info', 'message': 'Completed epoch 4/4', 'created_at': 1677995168}]}


In [123]:
finetune_event(ftId='ft-x4oK2iIg4uY3hoBZEVaaTtZm',api_key=my_key)

{'object': 'list', 'data': [{'object': 'fine-tune-event', 'level': 'info', 'message': 'Created fine-tune: ft-x4oK2iIg4uY3hoBZEVaaTtZm', 'created_at': 1677997477}]}


In [125]:
finetune_get(ftId='ft-x4oK2iIg4uY3hoBZEVaaTtZm',api_key=my_key)

{'object': 'fine-tune', 'id': 'ft-x4oK2iIg4uY3hoBZEVaaTtZm', 'hyperparams': {'n_epochs': 4, 'batch_size': 1, 'prompt_loss_weight': 0.01, 'learning_rate_multiplier': 0.1}, 'organization_id': 'org-QWWIpNe4Z3KsRVI0YR6lhyJW', 'model': 'davinci', 'training_files': [{'object': 'file', 'id': 'file-3FeGCNbpkIgtcOtatfOSYzdO', 'purpose': 'fine-tune', 'filename': 'file', 'bytes': 13403, 'created_at': 1677997340, 'status': 'processed', 'status_details': None}], 'validation_files': [], 'result_files': [{'object': 'file', 'id': 'file-KtcpnrZUymEwUMYhy5TM5Jlb', 'purpose': 'fine-tune-results', 'filename': 'compiled_results.csv', 'bytes': 4399, 'created_at': 1677997868, 'status': 'processed', 'status_details': None}], 'created_at': 1677997477, 'updated_at': 1677997869, 'status': 'succeeded', 'fine_tuned_model': 'davinci:ft-kamal:video-idea-generator-2023-03-05-06-31-07', 'events': [{'object': 'fine-tune-event', 'level': 'info', 'message': 'Created fine-tune: ft-x4oK2iIg4uY3hoBZEVaaTtZm', 'created_at': 

In [129]:
reply_dict_holder = []
for ind,pmt in enumerate(prompt_list):
    #print(prompt_list[ind])
    reply_dict = gpt3_completion(org=my_org,key=my_key,
                prompt=prompt_list[ind],
                   prompt_part=prompt_parts[ind],
                    filename='full_model_payload.jsonl')
    reply_dict_holder.append(reply_dict)

In [130]:
len(reply_dict_holder)

256

In [131]:
import pandas as pd

In [132]:
reply_dataframe = pd.DataFrame(reply_dict_holder)
reply_dataframe

prompt  \
0    Year: 2022\nPlace: India\nIdea1:Power\nIdea2:G...   
1    Year: 2022\nPlace: India\nIdea1:Power\nIdea2:e...   
2    Year: 2022\nPlace: India\nIdea1:Power\nIdea2:s...   
3    Year: 2022\nPlace: India\nIdea1:Power\nIdea2:s...   
4    Year: 2022\nPlace: India\nIdea1:Ability\nIdea2...   
..                                                 ...   
251  Year: 2028\nPlace: Russia\nIdea1:automation\nI...   
252  Year: 2028\nPlace: Russia\nIdea1:danger\nIdea2...   
253  Year: 2028\nPlace: Russia\nIdea1:danger\nIdea2...   
254  Year: 2028\nPlace: Russia\nIdea1:danger\nIdea2...   
255  Year: 2028\nPlace: Russia\nIdea1:danger\nIdea2...   

                                            completion  
0    by a production house.\n\n1. In a democracy, p...  
1    (1-2 minutes)\n\n1. In today's world, power is...  
2    .\n\n1. Power is the ability to influence or c...  
3    .\n\n1. In space faring, power is the ability ...  
4    and post on your youtube channel.\n\n\n1. At a...  
..                                                 ...  
251  .\n\n\n1. Space exploration has always been a ...  
252  presentation\n\n1. The dangers of government c...  
253  .\n\n1. The danger of not getting an education...  
254  .\n\nDangers can come in many different forms....  
255  for youtube.\n\nDanger in space is a very real...  

[256 rows x 2 columns]

In [138]:
reply_dataframe['completion_length'] = reply_dataframe.completion. \
                            apply(lambda x: len(x))
final_df = reply_dataframe[reply_dataframe.completion_length > 375]

In [139]:
final_df.shape

(184, 3)

In [141]:
final_df.drop('completion_length',inplace=True,axis=1)

In [151]:
with open('filtered_file.jsonl','w') as jso:
    for d in final_df.to_dict(orient='records'):
        json.dump(obj=d,fp=jso)
        jso.write('\n')

In [152]:
!cat filtered_file.jsonl

{"prompt": "Year: 2022\nPlace: India\nIdea1:Power\nIdea2:Government", "completion": "by a production house.\n\n1. In a democracy, power ultimately lies with the people. The government is responsible for ensuring that the will of the people is carried out and that their rights are protected.\n\n2. Government has the power to make laws and pass policies that can have a huge impact on the lives of its citizens.\n\n3. Government also has the power to enforce its laws and policies, and to use force if necessary.\n\n4. Government has the power to tax its citizens and to spend that money as it sees fit.\n\n5. Government has the power to borrow money, and to make decisions about how that money is to be used."}
{"prompt": "Year: 2022\nPlace: India\nIdea1:Power\nIdea2:education", "completion": "(1-2 minutes)\n\n1. In today's world, power is everything. It can make or break you.\n\n2. Power is what gives you the ability to make things happen.\n\n3. Education is the key to success. It opens doors 

In [153]:
file_upload(filename='filtered_file.jsonl')

{
  "bytes": 147793,
  "created_at": 1678000239,
  "filename": "file",
  "id": "file-r9vI0KULMmBUYOvTyEEmQb2j",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}


<File file id=file-r9vI0KULMmBUYOvTyEEmQb2j at 0x7f56d6cf6160> JSON: {
  "bytes": 147793,
  "created_at": 1678000239,
  "filename": "file",
  "id": "file-r9vI0KULMmBUYOvTyEEmQb2j",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}

In [155]:
finetune_model(fileId='file-r9vI0KULMmBUYOvTyEEmQb2j',
               suffix='new_video_gen_model')

{'object': 'fine-tune', 'id': 'ft-7DGxEQjWAbRJlfCZrt3wQQwn', 'hyperparams': {'n_epochs': 4, 'batch_size': None, 'prompt_loss_weight': 0.01, 'learning_rate_multiplier': None}, 'organization_id': 'org-QWWIpNe4Z3KsRVI0YR6lhyJW', 'model': 'davinci', 'training_files': [{'object': 'file', 'id': 'file-r9vI0KULMmBUYOvTyEEmQb2j', 'purpose': 'fine-tune', 'filename': 'file', 'bytes': 147793, 'created_at': 1678000239, 'status': 'processed', 'status_details': None}], 'validation_files': [], 'result_files': [], 'created_at': 1678000327, 'updated_at': 1678000327, 'status': 'pending', 'fine_tuned_model': None, 'events': [{'object': 'fine-tune-event', 'level': 'info', 'message': 'Created fine-tune: ft-7DGxEQjWAbRJlfCZrt3wQQwn', 'created_at': 1678000327}]}
